<a href="https://colab.research.google.com/github/BonTapHoa/paic-tho-san-hang-e/blob/main/test-12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline: IELTS Writing Task 1 Scoring

Chào mừng quý thí sinh đến với PTNK AI Challenge do The Noders Community tổ chức.

Trong notebook này, chúng ta sẽ xây dựng một mô hình baseline đơn giản để chấm điểm bài thi IELTS Writing Task 1. Mục tiêu là cung cấp một bộ mã nguồn cơ bản để thí sinh nộp kết quả và làm nền tảng cho các cải tiến tiếp theo. Nếu thí sinh là người mới hoàn toàn, vui lòng tham khảo https://docs.google.com/document/d/1545wNRM0TiUealIxDRkfDh8kfnjFATgWdsRPRZTRC0s/edit?usp=sharing để tìm hiểu các khái niệm liên quan.

### Mục tiêu
*   **Input**: Bài văn (Essay) của thí sinh.
*   **Output**: Điểm số (Band Score) từ 1.0 - 9.0.
*   **Đánh giá**: MAE (Mean Absolute Error). Giá trị nhỏ hơn biểu thị hiệu năng tốt hơn.

---
### Hướng dẫn dành cho người mới
Notebook này cung cấp một baseline; thí sinh được khuyến nghị phát triển và tối ưu hoá thêm. Ở mỗi phần có các gợi ý kỹ thuật. Vui lòng đọc kỹ các hướng dẫn này vì chúng giúp thí sinh cải thiện kết quả.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 1. Cài đặt & Import thư viện
# Chúng ta sẽ dùng thư viện Transformers của HuggingFace - tiêu chuẩn vàng trong NLP hiện nay.

import os
import sys
import warnings
import logging

# --- CẤU HÌNH ẨN CÁC WARNING KHÔNG CẦN THIẾT ---
# Tắt log của TensorFlow (nếu có) để giảm thông tin không cần thiết cho người mới
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
logging.getLogger("transformers").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cài đặt các thư viện cần thiết nếu chưa có
try:
    import transformers
    import datasets
    import gdown
except ImportError:
    print("Đang cài đặt các thư viện: transformers, datasets, accelerate, gdown, scikit-learn...")
    # Thêm -q để cài đặt im lặng hơn
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "transformers", "datasets", "accelerate", "gdown", "scikit-learn"])

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
import torch.nn as nn
from transformers import AutoModel, PreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


# Kiểm tra GPU (Nếu có GPU thì train sẽ nhanh hơn rất nhiều)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Thiết bị sử dụng: {device.upper()}")


Thiết bị sử dụng: CUDA


In [3]:
# 2. Tải dữ liệu từ Google Drive
# Link dữ liệu đã được cung cấp sẵn
train_file_id = '1VLdW_0J7OZdBOPlD-R__JUUCYfGLYpz9'
test_file_id = '1oZioLvy29aLzCZSYALnlYVLSitiQI5Tb'

def download_from_drive(file_id, output_name):
    url = f'https://drive.google.com/uc?id={file_id}'
    if not os.path.exists(output_name):
        print(f"Đang tải {output_name}...")
        gdown.download(url, output_name, quiet=False)
    else:
        print(f"{output_name} đã tồn tại.")

download_from_drive(train_file_id, 'train.csv')
download_from_drive(test_file_id, 'test.csv')

# Đọc dữ liệu
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print(f"Kích thước tập Train: {df_train.shape}")
print(f"Kích thước tập Test: {df_test.shape}")
display(df_train.head(3))

train.csv đã tồn tại.
test.csv đã tồn tại.
Kích thước tập Train: (4517, 9)
Kích thước tập Test: (897, 4)


,ID,prompt,image_description,essay,Overall,TR,CC,LR,GRA
0,0,The charts below show the proportions of Briti...,The image shows two pie charts comparing the p...,The presented pie charts display information a...,6.5,7.5,6.0,6.0,6.0
1,1,The plans below show a public park when it fir...,The image displays two maps of Grange Park sho...,The plans illustrate how Grange Park appeared ...,9.0,9.0,9.0,9.0,9.0
2,2,The graph shows the proportion of four differe...,"The image shows a line graph titled ""Recycling...",The line graph illustrates the percentages of ...,8.0,7.0,8.0,7.0,9.0


### Gợi ý 1: Dữ liệu đầu vào (Data Input)
Trong baseline này, chúng ta **chỉ sử dụng cột `Essay`** để huấn luyện mô hình.
Tuy nhiên, điểm số IELTS Writing Task 1 phụ thuộc nhiều vào việc mô tả đúng biểu đồ.
Gợi ý: Kết hợp cột `Prompt` (đề bài) và `Image Description` (mô tả ảnh) cùng `Essay` để mô hình nắm rõ ngữ cảnh.
Ví dụ: `Input = Prompt + " [SEP] " + Essay`

Chú thích: `SEP` là token phân tách giúp mô hình phân biệt các phần văn bản trong cùng một input.


In [4]:
# 3. Xử lý dữ liệu (Preprocessing) đơn giản
df_train['text'] = "[PROMPT] " + df_train['prompt'] + " [IMAGE DESC] " + df_train['image_description'] + " [ESSAY] " + df_train['essay']
df_test['text'] = "[PROMPT] " + df_test['prompt'] + " [IMAGE DESC] " + df_test['image_description'] + " [ESSAY] " + df_test['essay']

# Chỉ lấy cột Essay và Overall Score
# Chuẩn hóa tên cột cho dễ làm việc
df_train = df_train.rename(columns={'TR': 'label1', 'CC': 'label2', 'LR': 'label3', 'GRA': 'label4'})
cols_to_combine = ['label1', 'label2', 'label3', 'label4']
df_train['labels'] = df_train[cols_to_combine].values.tolist()

# Chia tập Train thành Train/Validation (80/20) để đánh giá model
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42) #test size 0.2 nghĩa là 20% là test

print(f"Train size: {len(train_df)}")
print(f"Val size: {len(val_df)}")

Train size: 3613
Val size: 904


### Gợi ý 2: Tokenization & Model
Baseline sử dụng **BERT Small** (`prajjwal1/bert-small`). Đây là mô hình kích thước nhỏ (~29 triệu tham số); thời gian chạy nhanh nhưng khả năng biểu diễn ngữ nghĩa hạn chế.
Gợi ý: Hãy thử đổi sang các mô hình mạnh hơn như:
*   `distilbert-base-uncased` (66M params)
*   `microsoft/deberta-v3-base` (86M params - SOTA hiện nay)
*   `roberta-base` (125M params)
Lưu ý: Các mô hình lớn hơn sẽ huấn luyện lâu hơn nhưng thường cho kết quả tốt hơn.


In [5]:
# 4. Tokenization & Dataset
# Sử dụng model nhỏ để làm baseline (dễ tiếp cận cho người mới)
model_checkpoint = "microsoft/deberta-v3-base" # ~29M params
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Hàm tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Chuyển Pandas DataFrame thành HuggingFace Dataset
train_dataset = datasets.Dataset.from_pandas(train_df[['text', 'labels']])
val_dataset = datasets.Dataset.from_pandas(val_df[['text', 'labels']])
test_dataset = datasets.Dataset.from_pandas(df_test[['text']])

# Áp dụng tokenize
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Xóa các cột text gốc để giải phóng bộ nhớ
tokenized_train = tokenized_train.remove_columns(["text", "__index_level_0__"])
tokenized_val = tokenized_val.remove_columns(["text", "__index_level_0__"])
tokenized_test = tokenized_test.remove_columns(["text"])

# Định dạng cho PyTorch
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids"])

print("Đã chuẩn bị xong dataset.")

Map:   0%|          | 0/3613 [00:00<?, ? examples/s]

Map:   0%|          | 0/904 [00:00<?, ? examples/s]

Map:   0%|          | 0/897 [00:00<?, ? examples/s]

Đã chuẩn bị xong dataset.


### 💡 Hint 3: Training Strategy
Trong ví dụ này chỉ huấn luyện **3 epochs** với các tham số mặc định.
👉 **Gợi ý:**
*   Tăng số epoch lên (5-10) và dùng `EarlyStopping` để tránh Overfitting.
*   Tinh chỉnh `learning_rate` (thường trong khoảng 2e-5 đến 5e-5).
*   Áp dụng **K-Fold Cross Validation** (chia dữ liệu thành 5 phần và huấn luyện 5 mô hình) để có kết quả ổn định hơn so với huấn luyện một lần.

(Phần này thuộc mức nâng cao; ban tổ chức chưa chuẩn bị tài liệu chi tiết về nội dung này. Thí sinh có thể tham khảo tài liệu tham khảo bên ngoài hoặc đặt câu hỏi trên kênh trao đổi của cuộc thi.)

Giải thích thêm cho những bạn chưa quen: sau khi chạy cell huấn luyện phía dưới, kết quả đánh giá sẽ hiển thị cột `MAE`. Với mô hình minh họa do ban tổ chức sử dụng, MAE tham khảo thường nằm trong khoảng 0.8–1.0. Giá trị này cho biết sai số trung bình của dự đoán (khoảng 0.8–1.0 band). Kết quả có thể được cải thiện nếu áp dụng các gợi ý ở trên hoặc thử nghiệm thêm các phương pháp nâng cao.

In [7]:
# 5. Khởi tạo Model & Trainer
# num_labels=1 vì đây là bài toán Regression (Hồi quy) - dự đoán 1 con số

# 1. Định nghĩa Class Model tùy chỉnh
class IELTSModel(nn.Module):
    def __init__(self, checkpoint, num_labels=1):
        super(IELTSModel, self).__init__()
        # Load phần thân BERT (pre-trained backbone)
        self.bert = AutoModel.from_pretrained(checkpoint)

        # Lấy kích thước vector đầu ra của BERT (thường là 768 với bert-base/small)
        hidden_size = self.bert.config.hidden_size

        # --- TÙY CHỈNH PHẦN HEAD TẠI ĐÂY ---
        # Kiến trúc: Linear -> BatchNorm -> ReLU -> Dropout -> Linear (Output)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),         # Lớp ẩn đầu tiên
            nn.BatchNorm1d(256),                 # [Regularization] Batch Norm giúp ổn định trọng số
            nn.ReLU(),                           # Hàm kích hoạt
            nn.Dropout(0.5),                     # [Regularization] Dropout 30% để chống Overfitting
            nn.Linear(256, num_labels)           # Lớp đầu ra
        )

        self.loss_fct = nn.MSELoss()

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        try:
            cls_output = outputs.pooler_output
        except:
            cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)

        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels.float())

        return SequenceClassifierOutput(loss=loss, logits=logits)

model = IELTSModel(model_checkpoint, num_labels=4)
model.to(device)

# Định nghĩa hàm tính metric MAE
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mae = mean_absolute_error(labels, predictions)
    return {"mae": mae}

# Cấu hình tham số huấn luyện
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/PTNK/PAIC/ielts_4",
    logging_strategy="epoch",
    eval_strategy="epoch", # Đánh giá sau mỗi epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    load_best_model_at_end=True,
    save_total_limit=2,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    metric_for_best_model="mae",
    greater_is_better=False, # MAE càng thấp càng tốt
    report_to="none" # Tắt log wandb
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

print("Bắt đầu huấn luyện...")
trainer.train()
print("Huấn luyện hoàn tất.")

Bắt đầu huấn luyện...


Epoch,Training Loss,Validation Loss,Mae
1,36.953700,29.002048,5.271399
2,24.644000,16.280989,3.895864
3,18.099400,39.368759,6.056951
4,12.259000,7.616001,2.505185
5,7.167600,3.259833,1.488020
6,4.925800,1.111914,0.815294
7,4.092400,1.452492,0.948266
8,4.747300,0.895571,0.714223
9,3.656100,1.130264,0.836928
10,3.395300,1.147645,0.824891


Huấn luyện hoàn tất.


### Gợi ý 4: Post-processing & Ensemble
Kết quả dự đoán của model là số thực (ví dụ: 6.342).
Tuy nhiên, điểm IELTS thường được làm tròn (ví dụ: 6.0, 6.5, 7.0).
Gợi ý:
*   Viết hàm làm tròn điểm về thang 0.5 gần nhất.
*   **Ensemble:** Train nhiều model khác nhau (DistilBERT, DeBERTa, RoBERTa) rồi lấy trung bình cộng kết quả dự đoán. Đây là cách hiệu quả nhất để giảm MAE!


In [8]:
import numpy as np

def pre_processing(arr):
    """
    Hàm xử lý hậu kỳ cho mô hình 4 labels (TR, CC, LR, GRA).
    Input: Mảng numpy kích thước (N, 4) hoặc (N*4,)
    Output: Mảng numpy kích thước (N,) chứa điểm Overall đã làm tròn.
    """
    # 1. Kiểm tra kích thước dữ liệu
    # Nếu đầu vào bị flatten (thành 1 chiều), ta reshape lại thành (Số mẫu, 4 cột)
    if arr.ndim == 1:
        # -1 nghĩa là tự tính số hàng, 4 là số cột cố định
        arr = arr.reshape(-1, 4)

    # 2. Tính điểm Overall = Trung bình cộng của 4 điểm thành phần
    # axis=1 nghĩa là tính trung bình theo từng hàng (từng bài thi)
    overall_raw = np.mean(arr, axis=1)

    # 3. Làm tròn về thang 0.5 (IELTS Band Score rounding)
    # Công thức: Nhân 2 -> Làm tròn số nguyên -> Chia 2
    # VD: 6.125 -> 12.25 -> 12 -> 6.0
    # VD: 6.35  -> 12.7  -> 13 -> 6.5
    y_pred = np.round(overall_raw * 2) / 2

    # 4. Cắt (Clip) điểm trong khoảng hợp lệ [1.0 - 9.0]
    # Đề phòng model dự đoán ra số âm hoặc > 9
    y_pred = np.clip(y_pred, 1.0, 9.0)

    return y_pred

# --- PHẦN TÍCH HỢP VÀO CODE CỦA BẠN ---

# 6. Dự đoán trên tập Test & Tạo Submission
print("Đang dự đoán trên tập test...")
predictions = trainer.predict(tokenized_test)

# LƯU Ý QUAN TRỌNG:
# Vì model giờ ra 4 cột, ta KHÔNG NÊN dùng .flatten() ngay ở đây.
# Hãy truyền trực tiếp predictions.predictions (dạng matrix) vào hàm xử lý.
y_pred = pre_processing(predictions.predictions)

# Lưu kết quả
if 'id' in df_test.columns:
    ids = df_test['id']
else:
    print("Không tìm thấy cột 'id' trong file test. Sử dụng index làm id...")
    ids = df_test.index

submission = pd.DataFrame({
    'id': ids,
    'Overall': y_pred
})

# Hiển thị vài mẫu để kiểm tra
print(submission.head())

# Lưu file csv
submission.to_csv('submission.csv', index=False)
print("Đã lưu file submission.csv. Vui lòng nộp file này để kiểm tra kết quả.")

Đang dự đoán trên tập test...


Không tìm thấy cột 'id' trong file test. Sử dụng index làm id...
   id  Overall
0   0      4.5
1   1      9.0
2   2      6.5
3   3      6.5
4   4      7.5
Đã lưu file submission.csv. Vui lòng nộp file này để kiểm tra kết quả.


In [ ]:
import os
import shutil

source_folder = '/content/ielts_2'
destination_base_path = '/content/drive/MyDrive/PTNK/PAIC/2'

# Ensure the base destination directory exists
if not os.path.exists(destination_base_path):
    os.makedirs(destination_base_path, exist_ok=True)

# Define the full destination path for the copied folder
destination_folder = os.path.join(destination_base_path, os.path.basename(source_folder))

try:
    # If the destination folder already exists, remove it before copying
    if os.path.exists(destination_folder):
        print(f"Thư mục đích '{destination_folder}' đã tồn tại. Đang xóa...")
        shutil.rmtree(destination_folder)
        print("Đã xóa thành công.")

    print(f"Đang sao chép nội dung từ '{source_folder}' đến '{destination_base_path}'...")
    shutil.copytree(source_folder, destination_folder)
    print(f"Đã sao chép nội dung từ '{source_folder}' đến '{destination_folder}' thành công.")
except Exception as e:
    print(f"Có lỗi xảy ra khi sao chép: {e}")

Đang sao chép nội dung từ '/content/ielts_2' đến '/content/drive/MyDrive/PTNK/PAIC/2'...
Đã sao chép nội dung từ '/content/ielts_2' đến '/content/drive/MyDrive/PTNK/PAIC/2/ielts_2' thành công.
